<a href="https://colab.research.google.com/github/swapnilsethi/Stat-5000/blob/main/House_Price_Predication_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import seaborn as sns

pd.pandas.set_option('display.max_columns',None)

In [3]:
#read data from github
URL = "https://raw.githubusercontent.com/swapnilsethi/Stat-5000/master/train.csv"

data = pd.read_csv(URL)

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
round((data.isnull().mean()).sort_values(ascending=False), 4) #Calculate % of missing values in a column


In [7]:
data = data.drop(['MiscFeature','PoolQC', 'Alley','Fence'], 1) #drop columns having 50% or more null values 

In [ ]:
# Relationship between null values and sales price.
features_with_na=[features for features in data.columns if data[features].isnull().sum()>1]

nulldata = data.copy()
color = iter(cm.rainbow(np.linspace(0, 1, 10)))
for feature in features_with_na:
    
    # 1 indicates observation was missing or zero otherwise
    nulldata[feature] = np.where(nulldata[feature].isnull(), 1, 0)
    
    #  mean SalePrice where the information is missing or present
    nulldata.groupby(feature)['SalePrice'].median().plot.bar()
    plt.title(feature)
    plt.show()

Here With the relation between the missing values and the dependent variable 
is clearly visible.So We need to replace these nan values with something meaningful which we will do in the Feature Engineering section

In [ ]:
data.dtypes

In [10]:
numerical_features = data.select_dtypes(exclude='object')
#Num_X = Num_X.drop('SalePrice', 1)

In [46]:
numerical_features.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,250000


In [ ]:
#Temporal Variables
temporal_feature = [feature for feature in numerical_features if 'Yr' in feature or 'Year' in feature]
temporal_feature

In [ ]:
for feature in temporal_feature:
    print(feature, data[feature].unique())

In [ ]:
#relation betwwen year sold and SalesPrice
data.groupby('YrSold')['SalePrice'].median().plot()
plt.xlabel('Year Sold')
plt.ylabel('Median house price')
plt.title('SalePrice vs YrSold')

In [ ]:
for feature in temporal_feature:
    if feature!='YrSold':
        dataset=data.copy()
        color = iter(cm.rainbow(np.linspace(0, 1, 3)))
        ## We will capture the difference between year variable and year the house was sold for
        dataset[feature]=dataset['YrSold']-dataset[feature]
        plt.scatter(dataset[feature],dataset['SalePrice'], c=next(color))
        plt.xlabel(feature)
        plt.ylabel('SalePrice')
        plt.show()

In [16]:
# discrete features 
discrete_feature = [feature for feature in numerical_features if len(data[feature].unique())<25 and feature not in temporal_feature+['Id']+['SalePrice']]


In [ ]:
data[discrete_feature].head()

In [ ]:
for feature in discrete_feature:
    dataset = data.copy()
    color = iter(cm.rainbow(np.linspace(0, 1, 18)))
    dataset.groupby(feature)['SalePrice'].median().plot.bar(color= next(color))
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title('Relationship between ' + feature + ' SalePrice')
    plt.show()

In [19]:
# continuous features 
continuous_feature = [feature for feature in numerical_features if feature not in temporal_feature+ discrete_feature +['Id']]


In [ ]:
for feature in continuous_feature:
    dataset = data.copy()
    color = iter(cm.rainbow(np.linspace(0, 1, 18)))
    dataset[feature].hist(bins=25, color = next(color))
    plt.xlabel(feature)
    plt.ylabel('count')
    plt.title( feature)
    plt.show()

In [ ]:
for feature in continuous_feature:
    dataset = data.copy()
    if 0 in dataset[feature].unique():
        pass
    else:
        dataset[feature]=np.log(dataset[feature])
    dataset.boxplot(column = feature)
    plt.title( feature)
    plt.show()

In [34]:
Categorical_feature = [feature for feature in data.columns if data[feature].dtypes=='object']
Categorical_feature

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [36]:
data[Categorical_feature].head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,NaN,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,Gd,Typ,Gd,Detchd,Unf,TA,TA,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal


In [45]:
for feature in Categorical_feature:
    print(feature +" " + str(len(data[feature].unique())))

MSZoning 5
Street 2
LotShape 4
LandContour 4
Utilities 2
LotConfig 5
LandSlope 3
Neighborhood 25
Condition1 9
Condition2 8
BldgType 5
HouseStyle 8
RoofStyle 6
RoofMatl 8
Exterior1st 15
Exterior2nd 16
MasVnrType 5
ExterQual 4
ExterCond 5
Foundation 6
BsmtQual 5
BsmtCond 5
BsmtExposure 5
BsmtFinType1 7
BsmtFinType2 7
Heating 6
HeatingQC 5
CentralAir 2
Electrical 6
KitchenQual 4
Functional 7
FireplaceQu 6
GarageType 7
GarageFinish 4
GarageQual 6
GarageCond 6
PavedDrive 3
SaleType 9
SaleCondition 6


In [ ]:
#Num_Y = data.select_dtypes(exclude='object')['SalePrice']

In [ ]:
#cormat = Num_X.corr()
#round(cormat,2)

In [ ]:
#sns.heatmap(cormat);